In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import joblib

# List of tickers
tickers = ["BNB-USD", "BTC-USD", "ETH-USD", "XRP-USD"]
lookback = 14  # From the tuned model

# Loop over each ticker
for ticker in tickers:
    print(f"\nProcessing {ticker}...")
    
    # Load processed data
    df = pd.read_csv(f"{ticker}_processed.csv", index_col="Date", parse_dates=True)

    # Features and target
    features = ["Open", "High", "Low", "Close", "Volume", "Lag1", "Lag7", "SMA7", "RSI14", "MACD", "Returns"]
    X = df[features].values
    y = df["Target"].values

    # Normalize the data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    # Create sequences
    X_seq, y_seq = [], []
    for i in range(lookback, len(X_scaled)):
        X_seq.append(X_scaled[i-lookback:i])
        y_seq.append(y_scaled[i])
    X_seq, y_seq = np.array(X_seq), np.array(y_seq)

    # Split the data (80% train, 20% test)
    train_size = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:train_size], X_seq[train_size:]
    y_train, y_test = y_seq[:train_size], y_seq[train_size:]

    # Build LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(lookback, X.shape[1])))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

    # Predict
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_test_actual = scaler_y.inverse_transform(y_test)

    # Metrics
    mse = mean_squared_error(y_test_actual, y_pred)
    mape = mean_absolute_percentage_error(y_test_actual, y_pred) * 100
    test_dates = df.index[lookback + train_size:]
    actual_series = pd.Series(y_test_actual.flatten(), index=test_dates)
    pred_series = pd.Series(y_pred.flatten(), index=test_dates)
    actual_direction = (actual_series.shift(-1) > actual_series).iloc[:-1].astype(int)
    predicted_direction = (pred_series.shift(-1) > pred_series).iloc[:-1].astype(int)
    directional_accuracy = (actual_direction == predicted_direction).mean() * 100

    print(f"MSE for {ticker}: {mse}")
    print(f"MAPE for {ticker}: {mape:.2f}%")
    print(f"Directional Accuracy for {ticker}: {directional_accuracy:.2f}%")

    # Save the model and scalers
    model.save(f"{ticker}_lstm_model.h5")
    joblib.dump(scaler_X, f"{ticker}_scaler_X.pkl")
    joblib.dump(scaler_y, f"{ticker}_scaler_y.pkl")
    print(f"Model and scalers saved for {ticker}")

    # Generate signals
    results = pd.DataFrame({
        "Close": df["Close"].iloc[lookback + train_size:],
        "Actual": actual_series,
        "Predicted": pred_series
    })
    results["Signal"] = 0  # Default: Hold
    threshold = 0.02  # 2% threshold
    results.loc[results["Predicted"] > results["Close"] * (1 + threshold), "Signal"] = 1  # Buy
    results.loc[results["Predicted"] < results["Close"] * (1 - threshold), "Signal"] = -1  # Sell

    # Save signals
    results.to_csv(f"{ticker}_signals.csv")
    print(f"\n{ticker} Signals Sample:")
    print(results[["Close", "Actual", "Predicted", "Signal"]].head())


Processing BNB-USD...
Epoch 1/50


C:\Users\Solent\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.1004 - val_loss: 0.0189
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0110 - val_loss: 0.0048
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0068 - val_loss: 0.0037
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0044 - val_loss: 0.0039
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0050 - val_loss: 0.0041
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0052 - val_loss: 0.0037
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0055 - val_loss: 0.0034
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0045 - val_loss: 0.0034
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0044 - val_loss: 0.0034
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0037 - val_loss: 0.0032
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0044 - val_loss: 0.0032
Epoch 12/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0039 - val_loss: 0.003

MSE for BNB-USD: 2532.533152146037
MAPE for BNB-USD: 6.29%
Directional Accuracy for BNB-USD: 45.65%
Model and scalers saved for BNB-USD

BNB-USD Signals Sample:
                 Close      Actual   Predicted  Signal
Date                                                  
2024-10-17  592.039612  599.364990  571.769409      -1
2024-10-18  599.364990  597.974915  573.626038      -1
2024-10-19  597.974915  605.517822  576.401489      -1
2024-10-20  605.517822  598.817566  577.738708      -1
2024-10-21  598.817566  593.360657  580.875366      -1

Processing BTC-USD...
Epoch 1/50


C:\Users\Solent\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.0484 - val_loss: 0.0039
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0119 - val_loss: 0.0033
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0113 - val_loss: 0.0037
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0078 - val_loss: 0.0036
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0089 - val_loss: 0.0055
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0070 - val_loss: 0.0040
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0058 - val_loss: 0.0038
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0059 - val_loss: 0.0040
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0060 - val_loss: 0.0072
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0064 - val_loss: 0.0034
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0055 - val_loss: 0.0030
Epoch 12/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0062 - val_loss

MSE for ETH-USD: 34398.869395601236
MAPE for ETH-USD: 4.98%
Directional Accuracy for ETH-USD: 45.65%
Model and scalers saved for ETH-USD

ETH-USD Signals Sample:
                  Close       Actual    Predicted  Signal
Date                                                     
2024-10-17  2604.273438  2641.552246  2666.838379       1
2024-10-18  2641.552246  2648.656982  2680.211914       0
2024-10-19  2648.656982  2746.364014  2701.731445       1
2024-10-20  2746.364014  2665.712158  2713.290527       0
2024-10-21  2665.712158  2620.197510  2764.006104       1

Processing XRP-USD...
Epoch 1/50


C:\Users\Solent\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.0051 - val_loss: 6.0770e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7.5611e-04 - val_loss: 2.6121e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5.2472e-04 - val_loss: 1.4867e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.5947e-04 - val_loss: 1.6480e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.5125e-04 - val_loss: 1.6414e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.3859e-04 - val_loss: 1.1768e-04
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.1552e-04 - val_loss: 1.1008e-04
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.6776e-04 - val_loss: 1.0153e-04
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.5605e-04 - val_loss: 1.0134e-04
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.4816e-04 - val_loss: 1.0089e-04
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.5384e-04 - val_loss: 9.8590

MSE for XRP-USD: 0.2652332993600259
MAPE for XRP-USD: 17.63%
Directional Accuracy for XRP-USD: 49.28%
Model and scalers saved for XRP-USD

XRP-USD Signals Sample:
               Close    Actual  Predicted  Signal
Date                                             
2024-10-17  0.543658  0.546541   0.539965       0
2024-10-18  0.546541  0.544257   0.541470       0
2024-10-19  0.544257  0.547652   0.542931       0
2024-10-20  0.547652  0.544863   0.541954       0
2024-10-21  0.544863  0.533379   0.542076       0
